# Functions for Gene Expressions

Define a function to normalize the expression data using a method called RPKM normalization.  This allows the comparison of measurements between different samples and genes.

<div class="alert alert-block alert-info">
In this case, we want ot highlight NumPy's vectorization and broadcasting rules, which allow us to manipulate and reason about data arrays very efficiently.
</div>

In [1]:
import numpy as np

In [2]:
def rpkm(counts, lengths):
    """Calculate reads per kilobase transcript per million reads.
    
    RPKM = (10^9 * C) / (N * L)
    
    Where:
    C = Number of reads mapped to a gene
    N = Total mapped reads in the experiment
    L = Exon length in base pairs for a gene
    
    Parameters
    ----------
    counts: array, shape (N_genes, N_samples)
        RNAseq (or similar) count data where columns are individual samples and rows are genes.
    lengths: array, shape (N_genes,)
        Gene lengths in base pairs in the same order as the rows in counts.
        
    Returns
    -------
    normed: array, shape (N_genes, N_samples)
        The RPKM normalized counts matrix.
    """
    
    N = np.sum(counts, axis=0)    # sum each column to get total reads per sample
    L = lengths
    C = counts
    
    normed = 1e9 * C / (N[np.newaxis, :] * L[:, np.newaxis])
    
    return(normed)

Make gene expression a list of lists.

In [3]:
gene0 = [100, 200]
gene1 = [50, 0]
gene2 = [350, 100]
expression_data = [gene0, gene1, gene2]

In [4]:
expression_data[2][0]

350

NumPy N-Dimensional Arrays

In [5]:
array1d = np.array([1,2,3,4])
print(array1d)
print(type(array1d))
print(array1d.shape)

[1 2 3 4]
<class 'numpy.ndarray'>
(4,)


In [6]:
array2d = np.array(expression_data)
print(array2d)
print(array2d.shape)
print(type(array2d))

[[100 200]
 [ 50   0]
 [350 100]]
(3, 2)
<class 'numpy.ndarray'>


<div class="alert alert-block alert-success">
This demonstrates that the shape function accounts for multi-dimensional arrays whereas len does not.
</div>

In [7]:
print(array2d.ndim)

2


## Why Use ndarrays Instead of Python Lists?

In [8]:
# Create an ndarray of integers in the range
# 0 up to (but not including) 1,000,000
array = np.arange(1e6)
list_array = array.tolist()

In [9]:
%timeit -n10 y = [val*5 for val in list_array]

186 ms ± 102 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%timeit -n10 x = array * 5

1.06 ms ± 228 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


<div class="alert alert-block alert-success">
Using numpy array makes the calculation 50x faster than a list comprehension on a list.
</div>

Ndarrays are more memory efficient than lists, using only the minimal memory needed for what is in the array.  You can use slices of arrays without making a copy.  If you modify this assignment it will change the original array.

In [11]:
# Create an ndarray x
x = np.array([1, 2, 3], np.int32)
print(x)

[1 2 3]


In [12]:
# Create a "slice" of x
y = x[:2]
print(y)

[1 2]


In [13]:
# Set the first element of y to be 6
y[0] = 6
print(y)

[6 2]


In [14]:
# Now the first element in x has changed to 6!
print(x)

[6 2 3]


<div class="alert alert-block alert-danger">
Notice that although we edited y, x has also changed, because y was referencing the same data!</div>

In [15]:
# You must be careful with array references, to make a copy
y = np.copy(x[:2])

## Vectorization

Apply a calculation to each element in an array without having to use a for loop.  This will also result in more readable code.

In [16]:
x = np.array([1, 2, 3, 4])
print(x * 2)

[2 4 6 8]


In [17]:
y = np.array([0, 1, 2, 1])
print(x + y)

[1 3 5 5]


## Broadcasting

Broadcasting is a way of performing implicit operations between two arrays, allowing you to perform operations on arrays of *compatible* shapes, to create arrays bigger than either of the starting ones.

In [18]:
x = np.array([1, 2, 3, 4])
x = np.reshape(x, (len(x), 1))
print(x)

[[1]
 [2]
 [3]
 [4]]


In [19]:
x.shape

(4, 1)

In [20]:
y = np.array([0, 1, 2, 1])
y = np.reshape(y, [1, len(y)])
print(y)

[[0 1 2 1]]


In [21]:
y.shape

(1, 4)

In [22]:
outer = x * y
print(outer)

[[0 1 2 1]
 [0 2 4 2]
 [0 3 6 3]
 [0 4 8 4]]


In [23]:
print(outer.shape)

(4, 4)


## Exploring a Gene Expressions Dataset

In [24]:
import pandas as pd

In [25]:
import bz2
filename = 'ElegantSciPy/data/counts.txt.bz2'

In [26]:
# Import TCGA malanoma data
with open(filename, 'rt') as f:
    data_table = pd.read_csv(filename, compression='bz2', header=0, sep=',', quotechar='"')

In [27]:
print(data_table.iloc[:5, :5])

  Unnamed: 0  00624286-41dd-476f-a63b-d2a5f484bb45  TCGA-FS-A1Z0  \
0       A1BG                               1272.36        452.96   
1       A1CF                                  0.00          0.00   
2      A2BP1                                  0.00          0.00   
3      A2LD1                                164.38        552.43   
4      A2ML1                                 27.00          0.00   

   TCGA-D9-A3Z1  02c76d24-f1d2-4029-95b4-8be3bda8fdbe  
0        288.06                                400.11  
1          0.00                                  1.00  
2          0.00                                  0.00  
3        201.83                                165.12  
4          0.00                                  0.00  


In [28]:
# Sample names
samples = list(data_table.columns)

In [29]:
filename = 'ElegantSciPy/data/genes.csv'
with open(filename, 'rt') as f:    # Parse file with pandas, index by GeneSymbol
    gene_info = pd.read_csv(f, index_col=0)
print(gene_info)

               GeneID  GeneLength
GeneSymbol                       
CPA1             1357        1724
GUCY2D           3000        3623
UBC              7316        2687
C11orf95        65998        5581
ANKMY2          57037        2611
C17orf81        23587        2399
TTC36          143941         814
IQCD           115811        2367
HSPA13           6782        3984
CD72              971        2480
MGMT             4255        1265
C19orf71    100128569         670
CEACAM19        56971        2246
FUNDC2          65991        2963
MRPL11          65003        1685
LOC440040      440040        2275
SLC7A14         57709       10133
MAP6             4135        5594
VPS54           51542        7416
TMED4          222068        2439
L1TD1           54596        3854
MEIG1          644890         625
CCDC136         64753        4659
C1orf92        149499        2183
ARSJ            79642        4759
PPIAL4C        653598         783
PAK1             5058        3657
FAM157A       

In [36]:
print("Genes in data_table: ", data_table.shape[0])
print("Genes in gene_info:  ", gene_info.shape[0])

Genes in data_table:  20500
Genes in gene_info:   20503


In [44]:
# Subset gene info to match the count data
matched_index = pd.Index.intersection(data_table.index, gene_info.index)

In [47]:
# 2D ndarray containing expression counts for each gene in each individual 
counts = np.asarray(data_table.loc[matched_index], dtype=int)

SyntaxError: invalid character in identifier (<ipython-input-47-0739e89aaa7a>, line 2)